In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "/content/drive/MyDrive/langchain-tutorial/Ch04. Advanced Rag/Data/투자설명서.pdf"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap = 100)

docs = loader.load_and_split(doc_splitter)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 데이터를 임베딩으로 변환
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS 벡터스토어 생성
faiss_store = FAISS.from_documents(docs, embedding)
# FAISS 벡터스토어 저장
persist_directory = "/content/DB"
faiss_store.save_local(persist_directory)

In [ ]:
# 저장한 FAISS DB 불러오기
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

In [ ]:
from pydantic import Field
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from sentence_transformers import CrossEncoder
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

In [ ]:
# ms-marco-MiniLM-L-12-v2 모델 다운로드
crossencoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [ ]:
class Retriever_with_cross_encoder(BaseRetriever):
    vectorstore: Any = Field(description="초기 검색을 위한 벡터 저장소")
    crossencoder: Any = Field(description="재순위화를 위한 크로스 인코더 모델")
    k: int = Field(default=5, description="초기에 검색할 문서 수")
    rerank_top_k: int = Field(default=2, description="재순위화 후 최종적으로 반환할 문서 수")

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        # 초기 검색
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)

        # 인코더용 쌍 준비
        pairs = [[query, doc.page_content] for doc in initial_docs]

        # 인코더 점수 획득
        scores = self.crossencoder.predict(pairs)

        # 점수별 문서 정렬
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)

        # 상위 재순위화 문서 반환
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

In [ ]:
# 크로스인코더 기반 리트리버 인스턴스 생성
cross_encoder_retriever = Retriever_with_cross_encoder(
    vectorstore=vectordb,
    crossencoder=crossencoder,
    k=4,  # 초기 밀집검색으로 반환할 문서 수를 설정
    rerank_top_k=2  # 리랭킹을 통해 최종적으로 반환할 문서 수를 설정
)

# 답변용 LLM 인스턴스 생성
llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o")

In [ ]:
# RetrievalQA 체인 인스턴스 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=cross_encoder_retriever,
    return_source_documents=True
)

In [ ]:
query = "이 회사의 2022년 영업손실이 정확히 얼마야?"
result = qa_chain({"query": query})

print(f"\n질문: {query}")
print(f"답변: {result['result']}")
print("\n답변 근거 문서:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content)  # Print each document